In [ ]:
import pandas as pd
BRCA_FR_SUPERFILTRATO_WG=pd.read_csv('/content/skca_br.txt',sep='\t')

In [ ]:
BRCA_FR_SUPERFILTRATO_WG

In [ ]:
ESONI=pd.read_csv('/content/drive/MyDrive/ESONI.tsv',sep='\t')

In [ ]:
BRCA_FR_SUPERFILTRATO_WG['chrom']='chr'+ BRCA_FR_SUPERFILTRATO_WG['chrom'].astype(str)

In [ ]:
ESONI['chrom']=ESONI['chrom'].astype(str)

In [ ]:
ESONI['exonStarts']=ESONI['exonStarts'].astype(int)

In [ ]:
ESONI['exonEnds']=ESONI['exonEnds'].astype(int)

In [ ]:
BRCA_FR_SUPERFILTRATO_WG['pos_start']=BRCA_FR_SUPERFILTRATO_WG['pos_start'].astype(int)
BRCA_FR_SUPERFILTRATO_WG['pos_end']=BRCA_FR_SUPERFILTRATO_WG['pos_end'].astype(int)

In [ ]:
import pandas as pd
import numpy as np
from multiprocessing import Pool

# Definizione della funzione per il confronto dei blocchi
def compare_blocks(block_BRCA, block_ESONI):
    # Resto del codice rimane invariato
    merged_block = block_BRCA.merge(block_ESONI, on='chrom', how='inner')
    filtered_block = merged_block[
        (merged_block['pos_start'] >= merged_block['exonStarts']) &
        (merged_block['pos_end'] <= merged_block['exonEnds'])
    ]
    return filtered_block[['Project', 'Sample', 'ID', 'Genome', 'mut_type', 'chrom', 'pos_start',
       'pos_end', 'ref', 'alt', 'Type']]

# Dimensione dei blocchi
block_size = 1000

# Calcolo del numero di blocchi per ciascun DataFrame
num_blocks_BRCA = int(np.ceil(len(BRCA_FR_SUPERFILTRATO_WG) / block_size))
num_blocks_ESONI = int(np.ceil(len(ESONI) / block_size))

# Creazione della lista di indici per i blocchi
block_indices = []
for i in range(num_blocks_BRCA):
    start_idx_BRCA = i * block_size
    end_idx_BRCA = min((i + 1) * block_size, len(BRCA_FR_SUPERFILTRATO_WG))

    for j in range(num_blocks_ESONI):
        start_idx_ESONI = j * block_size
        end_idx_ESONI = min((j + 1) * block_size, len(ESONI))

        block_indices.append([start_idx_BRCA, end_idx_BRCA, start_idx_ESONI, end_idx_ESONI])

# Funzione wrapper per la chiamata a compare_blocks con gli indici
def compare_blocks_wrapper(indices):
    block_BRCA = BRCA_FR_SUPERFILTRATO_WG.iloc[indices[0]:indices[1]]
    block_ESONI = ESONI.iloc[indices[2]:indices[3]]

    return compare_blocks(block_BRCA, block_ESONI)

# Utilizzo di multiprocessing per eseguire operazioni parallele
with Pool() as pool:
    results = pool.map(compare_blocks_wrapper, block_indices)

# Combinazione dei risultati in un unico DataFrame
final_result = pd.concat(results, ignore_index=True)

In [ ]:
final_result

In [ ]:
for i in range(final_result.shape[0]):
  final_result.loc[i,'chrom']=final_result.loc[i,'chrom'][3:]

In [ ]:
final_result

In [ ]:
final_result.to_csv('skca_br_we_input.txt', sep='\t', index=False)

In [ ]:
from google.colab import files
files.download('skca_br_we_input.txt')